In [24]:
using DelimitedFiles
using StatsBase 
using Random
using LinearAlgebra
using Plots
using LaTeXStrings
using Printf
rng = Random.MersenneTwister(1234);

In [25]:
include("../src/WF/MPL_Tools.jl")
include("../src/WF/MPL_run_binary.jl")
include("../src/WF/Bezier_interpolation_binary_filter.jl")
include("../src/WF/functions_performance_check.jl");
include("../src/common/common_Bezier.jl");
include("../src/common/common_analysis.jl");

# Make trajectory file for the Wright-Fisher model

In [12]:
file_key1 =  "allele-traject_id-"
data_dir = "../data/WF/"
file_key2 = "_mu-0.001_N-1000_WF_standard.txt"
dir_out = "../out/WF/"


L = 50
N = 1000;
time_upto = 300;

In [13]:
# This cell should be enclosed. 
for id_ensemble in 1:10
    (data, time_list) = get_data_time(file_key1, file_key2, data_dir, id_ensemble, time_upto)
    time_list = unique(time_list) 
    n_time = length(time_list)
    mean_x1 = zeros(n_time,L)
    mean_x1 = zeros(n_time, L)

    for id_t in 1:n_time
        t = time_list[id_t]
        (n_t, sample_t) = get_sample_at_t(data, t); # without interpolate
        (x1_t1, x2_t1) = get_x1_x2(L, n_t, sample_t)
        mean_x1[id_t, :] = copy(x1_t1)
    end    
    fout = open(dir_out * "WF_trajectories/WF-trajectory_id-"*string(id_ensemble)*".txt", "w");
    for id_t in 1:n_time
        for i in 1:L 
            print(fout, @sprintf("%.5f ", mean_x1[id_t, i]) )
        end
        println(fout, "")
    end
    close(fout);
end;

# Infer seleciton coefficients for sequences generated from Wright-Fisher models 

In [30]:
mu = 1e-3
time_steps = [1, 10, 30, 60, 75, 80, 100, 95]
gamma_set = [0, 1e-3, 0.1, 0.5, 1, 2, 5, 10, 30]
time_cutoff = 300
n_ensemble = 100 
N_sub_population = 750;

In [31]:
# This cell compute selection coefficients for 1000 enesembles and estimate probability denstiy of selection. 
# The total computation tims is 1300 sec.
n_time_steps = size(time_steps,1)
n_gamma_max = size(gamma_set,1)
selections_B = zeros(n_time_steps, n_ensemble, n_gamma_max, L)
selections_L = zeros(n_time_steps, n_ensemble, n_gamma_max, L)
selections_naive = zeros(n_time_steps, n_ensemble, n_gamma_max, L)

selections_B_SL = zeros(n_time_steps, n_ensemble, n_gamma_max, L)
selections_L_SL = zeros(n_time_steps, n_ensemble, n_gamma_max, L)
selections_naive_SL = zeros(n_time_steps, n_ensemble, n_gamma_max, L)

eigen_min_B = zeros(n_time_steps, n_ensemble)
eigen_min_L = zeros(n_time_steps, n_ensemble)
eigen_min_naive = zeros(n_time_steps, n_ensemble)

errorbar_Bez = zeros(n_time_steps, n_ensemble, L)
errorbar_Lin = zeros(n_time_steps, n_ensemble, L)

#----- enhance genetic drift. Turning on this function takes a long computation ---- #
### Bezier has advantage when large genetic drift presens, a small sub population size.
flag_large_genetic_drift = true
#-----------------------------

Population_tot = copy(N)

for n in 1:n_ensemble     
    for id_time_steps in 1:n_time_steps
        #----------- obtaining data ---------------#
        t_sampling = time_steps[id_time_steps]
        #fname_in = "../data/WF/allele-traject-time-1_id-" *string(n)* "_mu-0.001_N-1000_standard.txt"
        #fname_in = "../data/WF/allele-traject_id-" *string(n)* "_mu-0.001_N-1000_WF_standard.txt"
        fname_in = data_dir * file_key1 *string(n)* file_key2
        # this is test
        #fname_in = "../data_temp/WF/N100/allele-traject_id-" *string(n)* "_mu-0.001_N-100_WF_standard.txt"
        
        data = readdlm(fname_in);
        read_upto = count(data[:,1] .<= time_cutoff)
        data = copy(data[1:read_upto, :])
        
        ## selecting samples those collecting times are each t steps, 0, 30, 60, etc.
        selecting_indices = collect(1:size(data,1))[data[:,1] .% t_sampling .== 0]
        data = data[selecting_indices,:];
        
        ## resumpling from the large population
        if(flag_large_genetic_drift)
            data = resumpling_data(data, N, N_sub_population)
        end
        
        time_list = get_time_list(data);
        L = size(data,2)-3
        unique_time = unique(time_list);
        
        #------------ obtaining covariances and drifts -----------#
        #(C_tot_B, drift_B, x1_mean_B, x1_init_B, x1_fini_B, point_set_x1_a_B, point_set_x1_b_B, point_set_x2_a_B, point_set_x2_b_B) = get_Bezier_Ctot_drift_x1mean_binary_simple(L, data, unique_time);
        time_interval_threshold = 90
        (C_tot_B, drift_B, x1_mean_B, x1_init_B, x1_fini_B, point_set_x1_a_B, point_set_x1_b_B, point_set_x2_a_B, point_set_x2_b_B) = get_Bezier_Ctot_drift_x1mean_binary_simple_with_midpoints(L, data, unique_time, time_interval_threshold);
        
        (C_tot_L, drift_L, x1_mean_L, x1_init_L, x1_fini_L) = get_integrated_Ctot_drift_x1mean_binary_simple(L, data, unique_time);
        (C_tot_naive, drift_naive, x1_mean_naive, x1_init_naive, x1_fini_naive) = get_naive_Ctot_drift_x1mean(L, data, unique_time)
        
        errorbar_Bez[id_time_steps, n, :] = copy(C_tot_B[diagind(C_tot_B)])
        errorbar_Lin[id_time_steps, n, :] = copy(C_tot_L[diagind(C_tot_L)])
        
        eigen_min_B[id_time_steps, n] = minimum(eigen(C_tot_B).values)
        eigen_min_L[id_time_steps, n] = minimum(eigen(C_tot_L).values)
        eigen_min_naive[id_time_steps, n] = minimum(eigen(C_tot_naive).values)
        
        for n_gamma in 1:n_gamma_max
            gamma = gamma_set[n_gamma]
            
            #----------- obtaining the seelctions. ---------------------#            
            selections_B_temp = selection_coeff(mu, drift_B, C_tot_B + gamma*I, x1_init_B, x1_fini_B)
            selections_L_temp = selection_coeff(mu, drift_L, C_tot_L + gamma*I, x1_init_L, x1_fini_L);
            selections_naive_temp = selection_coeff(mu, drift_naive, C_tot_naive + gamma*I, x1_init_naive, x1_fini_naive);
            #----------- register the estimated selections  ------------#
            selections_B[id_time_steps, n, n_gamma, :] = copy(selections_B_temp)
            selections_L[id_time_steps, n, n_gamma, :] = copy(selections_L_temp)
            selections_naive[id_time_steps, n, n_gamma, :] = copy(selections_naive_temp)
            #----------- for single locus (SL) methods -----------------#
            
            C_tot_B_SL = copy(diagm(0=>C_tot_B[diagind(C_tot_B)]))
            C_tot_L_SL = copy(diagm(0=>C_tot_L[diagind(C_tot_L)]))
            C_tot_naive_SL = copy(diagm(0=>C_tot_naive[diagind(C_tot_naive)]))
            #----------- obtaining the seelctions. ---------------------#            
            selections_B_temp_SL = selection_coeff(mu, drift_B, C_tot_B_SL + gamma*I, x1_init_B, x1_fini_B)
            selections_L_temp_SL = selection_coeff(mu, drift_L, C_tot_L_SL + gamma*I, x1_init_L, x1_fini_L);
            selections_naive_temp_SL = selection_coeff(mu, drift_naive, C_tot_naive_SL + gamma*I, x1_init_naive, x1_fini_naive);
            #----------- register the estimated selections  ------------#
            selections_B_SL[id_time_steps, n, n_gamma, :] = copy(selections_B_temp_SL)
            selections_L_SL[id_time_steps, n, n_gamma, :] = copy(selections_L_temp_SL)
            selections_naive_SL[id_time_steps, n, n_gamma, :] = copy(selections_naive_temp_SL)

        end
    end
end
        

## Output

In [38]:
# Output selections 
# This is temp, ouput also, diagonal covariance elements for errobars

fout_selections_Bez = open(dir_out * "selections_multiple_Dt_reg_cases_Bez.dat", "w")
fout_selections_Lin = open(dir_out * "selections_multiple_Dt_reg_cases_Lin.dat", "w");
fout_selections_naive = open(dir_out * "selections_multiple_Dt_reg_cases_naive.dat", "w");

fout_errorbar_Bez = open(dir_out * "error_bar_multiple_Dt_reg_cases_Bez.dat", "w")
fout_errorbar_Lin = open(dir_out * "error_bar_multiple_Dt_reg_cases_Lin.dat", "w")

#------- Output estimated selections and errorbars --------#
for n_t in 1:n_time_steps
for n_reg in 1:n_gamma_max
for n in 1:n_ensemble
for i in 1:L
    println(fout_selections_Lin, n_t, " ", time_steps[n_t], " ", n_reg,  " ", gamma_set[n_reg], " ", 
                    n, " ", i, " ", 
                    selections_L[n_t, n, n_reg, i], " ", selections_L_SL[n_t, n, n_reg, i])
    println(fout_selections_Bez, n_t, " ", time_steps[n_t], " ", n_reg,  " ", gamma_set[n_reg], " ", 
                    n, " ", i, " ", 
                    selections_B[n_t, n, n_reg, i], " ", selections_B_SL[n_t, n, n_reg, i])
                
    println(fout_selections_naive, n_t, " ", time_steps[n_t], " ", n_reg,  " ", gamma_set[n_reg], " ", 
                    n, " ", i, " ", 
                    selections_naive[n_t, n, n_reg, i], " ", selections_naive_SL[n_t, n, n_reg, i])                
    
    if(n_reg==1)
        #----- errobrbar ---- #
        println(fout_errorbar_Lin, n_t, " ", time_steps[n_t], " ", 
                            n, " ", i, " ",
                            errorbar_Lin[n_t, n, i])
        println(fout_errorbar_Bez, n_t, " ", time_steps[n_t], " ", 
                            n, " ", i, " ", 
                            errorbar_Bez[n_t, n, i])                
    end
end
end
end
end
close(fout_selections_Lin); close(fout_selections_Bez); close(fout_selections_naive);
close(fout_errorbar_Bez); close(fout_errorbar_Lin);

#------- Output eivenvalue distributions --------#
fout_eigenvalue_Bez = open(dir_out * "eigenvalues_cov_Bez.dat", "w")
fout_eigenvalue_Lin = open(dir_out * "eigenvalues_cov_Lin.dat", "w");
fout_eigenvalue_naive = open(dir_out * "eigenvalues_cov_naive.dat", "w");
 
for n_t in 1:n_time_steps
    for n in 1:n_ensemble
        print(fout_eigenvalue_Lin, eigen_min_L[n_t, n], " ")
        print(fout_eigenvalue_Bez, eigen_min_B[n_t, n], " ")
        print(fout_eigenvalue_naive, eigen_min_naive[n_t, n], " ")
    end
    println(fout_eigenvalue_Lin, "")
    println(fout_eigenvalue_Bez, "")
    println(fout_eigenvalue_naive, "")
end
close(fout_eigenvalue_Lin); close(fout_eigenvalue_Bez); close(fout_eigenvalue_naive);
        

# Sampling interval, Δt dependncy on covarinace interpolation erorr.

In [33]:
# This is fore creating evolution of covarinaces 
n_time_steps = size(time_steps,1)
covariance_error_L = zeros(6, n_time_steps, n_ensemble) 
covariance_error_B = zeros(6, n_time_steps, n_ensemble)


@time for n in 1:n_ensemble     
    #---- computation of the integrated covariance for Δt==1 ------#
    #fname_in = "../data/WF/allele-traject-time-1_id-" *string(n)* "_mu-0.001_N-1000_standard.txt"

    fname_in = data_dir * file_key1 *string(n)* file_key2
    #test
    #fname_in = "../data_temp/WF/N100/allele-traject_id-" *string(n)* "_mu-0.001_N-100_WF_standard.txt"
    data = readdlm(fname_in);
    read_upto = count(data[:,1] .<= time_cutoff)
    data = copy(data[1:read_upto, :])

    time_list = get_time_list(data);
    L = size(data,2)-3
    N = size(data,1)
    unique_time = unique(time_list);
    
    (C_tot_B_0, drift_B, x1_mean_B, x1_init_B, x1_fini_B, point_set_x1_a_B, point_set_x1_b_B, point_set_x2_a_B, point_set_x2_b_B) = get_Bezier_Ctot_drift_x1mean_binary_simple(L, data, unique_time);
    time_interval_threshold = 90
    #(C_tot_B_0, drift_B_0, x1_mean_B, x1_init_B, x1_fini_B, point_set_x1_a_B, point_set_x1_b_B, point_set_x2_a_B, point_set_x2_b_B) = get_Bezier_Ctot_drift_x1mean_binary_simple_with_midpoints(L, data, unique_time, time_interval_threshold);
    (C_tot_L_0, drift_L_0, x1_mean_L, x1_init_L, x1_fini_L) = get_integrated_Ctot_drift_x1mean_binary_simple(L, data, unique_time);    
    
    # compute the normalization terms.
    norms_L_0 = get_norm_measures(C_tot_L_0)
    norms_B_0 = get_norm_measures(C_tot_B_0)
    
    for id_time_steps in 1:n_time_steps
        #----------- obtaining data ---------------#
        t_sampling = time_steps[id_time_steps]
        #fname_in = "../data/WF/allele-traject-time-1_id-" *string(n)* "_mu-0.001_N-1000_standard.txt"
        fname_in = data_dir * file_key1 *string(n)* file_key2
        
        #fname_in = "../data_temp/WF/N100/allele-traject_id-" *string(n)* "_mu-0.001_N-100_WF_standard.txt"
        data = readdlm(fname_in);
        read_upto = count(data[:,1] .<= time_cutoff)
        data = copy(data[1:read_upto, :])
        
        ## selecting samples those collecting times are each t steps, 0, 30, 60, etc.
        selecting_indices = collect(1:size(data,1))[data[:,1] .% t_sampling .== 0]
        data = data[selecting_indices,:];
                
        time_list = get_time_list(data);
        L = size(data,2)-3
        N = size(data,1)
        unique_time = unique(time_list);
        
        #------------ obtaining covariances and drifts -----------#
        (C_tot_B, drift_B, x1_mean_B, x1_init_B, x1_fini_B, point_set_x1_a_B, point_set_x1_b_B, point_set_x2_a_B, point_set_x2_b_B) = get_Bezier_Ctot_drift_x1mean_binary_simple(L, data, unique_time);
        #(C_tot_B, drift_B, x1_mean_B, x1_init_B, x1_fini_B, point_set_x1_a_B, point_set_x1_b_B, point_set_x2_a_B, point_set_x2_b_B) = get_Bezier_Ctot_drift_x1mean_binary_simple_with_midpoints(L, data, unique_time, time_interval_threshold);
        (C_tot_L, drift_L, x1_mean_L, x1_init_L, x1_fini_L) = get_integrated_Ctot_drift_x1mean_binary_simple(L, data, unique_time);
        
        norms_L = get_norm_measures(C_tot_L - C_tot_L_0)
        norms_B = get_norm_measures(C_tot_B - C_tot_B_0)
        
        covariance_error_L[:, id_time_steps, n] = norms_L ./ norms_L_0 
        covariance_error_B[:, id_time_steps, n] = norms_B ./ norms_B_0 
    end
end

1369.020575 seconds (11.80 G allocations: 4.489 TiB, 32.09% gc time)


In [34]:
# Get mean and standard deviation values.
covariance_error_L_mean = zeros(6, n_time_steps)
covariance_error_L_std = zeros(6, n_time_steps)
covariance_error_B_mean = zeros(6, n_time_steps)
covariance_error_B_std = zeros(6, n_time_steps)
for k in 1:6
    for n_t in 1:n_time_steps
        covariance_error_L_mean[k, n_t] = summarystats(covariance_error_L[k, n_t, :]).mean
        covariance_error_L_std[k, n_t] = std(covariance_error_L[k, n_t, :]) / sqrt(n_ensemble)        
        covariance_error_B_mean[k, n_t] = summarystats(covariance_error_B[k, n_t, :]).mean
        covariance_error_B_std[k, n_t] = std(covariance_error_B[k, n_t, :]) / sqrt(n_ensemble)
    end
end

In [35]:
# Output covariance interpolation errors
fout_cov_error_Lin_mean = open(dir_out * "covariance_error_Lin_mean.dat", "w");
fout_cov_error_Lin_std = open(dir_out * "covariance_error_Lin_std.dat", "w");
fout_cov_error_Bez_mean = open(dir_out * "covariance_error_Bez_mean.dat", "w")
fout_cov_error_Bez_std = open(dir_out * "covariance_error_Bez_std.dat", "w");

i_max, j_max = size(covariance_error_L_mean)
for i in 1:i_max
    for j in 1:j_max
        print(fout_cov_error_Lin_mean, covariance_error_L_mean[i,j], " ")
        print(fout_cov_error_Lin_std, covariance_error_L_std[i,j], " ")
        print(fout_cov_error_Bez_mean, covariance_error_B_mean[i,j], " ")
        print(fout_cov_error_Bez_std, covariance_error_B_std[i,j], " ")
    end
    println(fout_cov_error_Lin_mean)
    println(fout_cov_error_Lin_std)
    println(fout_cov_error_Bez_mean)
    println(fout_cov_error_Bez_std)
    
end
close(fout_cov_error_Lin_mean)
close(fout_cov_error_Lin_std)
close(fout_cov_error_Bez_mean)
close(fout_cov_error_Bez_std);

# Autocorrelations for the pairwise frequenceis for the Wright-Fisher model (this computation takes long time)

In [39]:
##### The following computation takes around 3-4 hours ####
T=300
N_ensemble=50
##### compute the equilibrium states. ############### 
t_start = 1

C_av_tot = zeros(T, L,L); 
C_t0_tot = zeros(N_ensemble, L,L); 
scale_N = 1.0/N_ensemble
C_var_tot = zeros(T, L, L)

@time for t in t_start:T
	C_temp = zeros(N_ensemble, L, L)	
	for id_ensemble in 1:N_ensemble
        fname_in = data_dir * file_key1 * string(id_ensemble) * file_key2
	    data = readdlm(fname_in)
	    
	    # keep the initial single site freq. and covariance.     
	    if(t==t_start) 
	        (n_t0, sample_t0) = get_sample_at_t(data, t)
	        (x1_t0, x2_t0, C_t0) = get_x1_x2_C(L, n_t0, sample_t0);
	        C_t0_tot[id_ensemble, :, :] = C_t0
	    end       
	    #--- computation of the averaged single site freq. and covariance.  
	    time_list = get_time_list(data)
	    (n_t, sample_t) = get_sample_at_t(data, t)
	    (x1_t, x2_t, C_t) = get_x1_x2_C(L, n_t, sample_t);
	    C_av_tot[t,:,:] += C_t
        C_temp[id_ensemble, :, :] = copy(C_t)
	end
    C_av_tot[t,:,:] *= scale_N 

	for i in 1:L
		for j in i:L
            C_var_tot[t, i, j] = std(C_temp[:, i, j])	
            C_var_tot[t, j, i] = C_var_tot[t, i, j]
		end
	end
end
#########################################################

#-------------------------------------------------------#
##### The following computation takes around 3 hours ####
############ compute the AutoC_t50orrelations   #########
t_start = 50
AutoC_t50 = zeros(2, T)
t0 = t_start 
@time for t in t_start:T
    AC_C = zeros(L,L); 
	AC_x2 = zeros(L,L); 
	AC_x1 = zeros(L)
	for id_ensemble in 1:N_ensemble
        fname_in = data_dir * file_key1 * string(id_ensemble) * file_key2
	    data = readdlm(fname_in)
	    time_list = get_time_list(data)
	    (n_t, sample_t) = get_sample_at_t(data, t)
	    (x1_t, x2_t, C_t) = get_x1_x2_C(L, n_t, sample_t);	
	    C_t0 = C_t0_tot[id_ensemble, :, :] 
	    
        AC_C += C_t .* C_t0 
	end
    AC_C  = copy(AC_C)  * scale_N - C_av_tot[t,:,:] .* C_av_tot[t0,:,:]
	# normalized by the 
	#AC_C  = copy(AC_C)  ./ ( C_var_tot[t0, :,:]  .* C_var_tot[t0, :,:] .+ 1e-3 ) 

	AC_diag_C = AC_C[diagind(AC_C)]	
	AC_offdiag_C = copy(AC_C)
 	AC_offdiag_C[diagind(AC_offdiag_C)] .= 0	

    AutoC_t50[1, t] = sum(AC_diag_C) / L 
	AutoC_t50[2, t] = sum(AC_offdiag_C)/ ( L*(L-1) )         		
end


11223.983517 seconds (171.77 G allocations: 5.433 TiB, 25.58% gc time)
9348.296850 seconds (143.72 G allocations: 4.550 TiB, 25.63% gc time)


In [40]:
# Output autocorrelation files. 
fout_acf = open(dir_out * "autocorrelation_WF.dat", "w");
for i in 51:size(AutoC_t50,2)
    for j in 1:(size(AutoC_t50,1)-1)
        print(fout_acf, AutoC_t50[j,i], " ")
    end
    println(fout_acf, AutoC_t50[end,i] )
end
close(fout_acf)